## Data Source
- [DOB Job Application Filings](https://data.cityofnewyork.us/Housing-Development/DOB-Job-Application-Filings/ic3t-wcy2/about_data)
- [NYC Citywide Annualized Calendar Sales Update](https://data.cityofnewyork.us/City-Government/NYC-Citywide-Annualized-Calendar-Sales-Update/w2pb-icbu/about_data)
- [DOB NOW: Build – Job Application Filings](https://data.cityofnewyork.us/Housing-Development/DOB-NOW-Build-Job-Application-Filings/w9ak-ipjd/about_data)

## Read Data

In [1]:
import pandas as pd
from sodapy import Socrata

client = Socrata("data.cityofnewyork.us", None)
df_list = []
offset = 0

while len(df_list)<50000:
    results = client.get("w2pb-icbu", 
                         limit=2000,
                         offset=offset)
    offset += 2000
    df_list.extend(results)

df_land = pd.DataFrame.from_records(df_list)

## Explore Data

In [2]:
df_land = df_land[
    ['borough',
     'neighborhood', 
     'building_class_category', 
     'zip_code',
     'land_square_feet',
     'gross_square_feet',
     'year_built',
     'sale_price',
     'sale_date',
     'latitude',
     'longitude']
    ]
df_land.head()

,borough,neighborhood,building_class_category,zip_code,land_square_feet,gross_square_feet,year_built,sale_price,sale_date,latitude,longitude
0,1,CHELSEA,21 OFFICE BUILDINGS,10001,"7,406","40,926",1926,43300000.00,2019-03-28T00:00:00.000,40.749704,-74.00493
1,1,CHELSEA,21 OFFICE BUILDINGS,10001,"9,890","83,612",1917,148254147.00,2019-05-23T00:00:00.000,40.749364,-74.004132
2,1,CHELSEA,21 OFFICE BUILDINGS,10001,"2,498","7,380",1910,11000000.00,2019-03-13T00:00:00.000,40.752067,-74.002931
3,1,CHELSEA,21 OFFICE BUILDINGS,10011,"34,188","281,361",1936,591800000.00,2019-05-22T00:00:00.000,NaN,NaN
4,1,CHELSEA,21 OFFICE BUILDINGS,10011,"2,469","5,603",1900,0.00,2019-04-01T00:00:00.000,40.745809,-73.999729


In [3]:
df_land['building_class_category'].unique()

array(['21 OFFICE BUILDINGS', '22 STORE BUILDINGS', '23 LOFT BUILDINGS',
       '30 WAREHOUSES', '31 COMMERCIAL VACANT LAND',
       '33 EDUCATIONAL FACILITIES', '44 CONDO PARKING', '45 CONDO HOTELS',
       '46 CONDO STORE BUILDINGS', '47 CONDO NON-BUSINESS STORAGE',
       '02 TWO FAMILY DWELLINGS', '07 RENTALS - WALKUP APARTMENTS',
       '08 RENTALS - ELEVATOR APARTMENTS',
       '10 COOPS - ELEVATOR APARTMENTS', '12 CONDOS - WALKUP APARTMENTS',
       '13 CONDOS - ELEVATOR APARTMENTS',
       '15 CONDOS - 2-10 UNIT RESIDENTIAL',
       '16 CONDOS - 2-10 UNIT WITH COMMERCIAL UNIT', '27 FACTORIES',
       '29 COMMERCIAL GARAGES', '43 CONDO OFFICE BUILDINGS',
       '17 CONDO COOPS', '05 TAX CLASS 1 VACANT LAND',
       '14 RENTALS - 4-10 UNIT', '09 COOPS - WALKUP APARTMENTS',
       '26 OTHER HOTELS', '32 HOSPITAL AND HEALTH FACILITIES',
       '34 THEATRES', '35 INDOOR PUBLIC AND CULTURAL FACILITIES',
       '01 ONE FAMILY DWELLINGS', '03 THREE FAMILY DWELLINGS',
       '25 LUXURY 

In [4]:
df_land['building_class_category'].nunique()

45

In [3]:
print(df_land['sale_date'].max())
print(df_land['sale_date'].min())

2020-12-27T00:00:00.000
2019-01-01T00:00:00.000


In [3]:
df_land.groupby('borough').size().sort_values(ascending=False)

borough
3            22982
1            16598
2             8138
4             2181
5               81
MANHATTAN       17
BRONX            3
dtype: int64

In [4]:
df_land.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 11 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   borough                  50000 non-null  object
 1   neighborhood             50000 non-null  object
 2   building_class_category  50000 non-null  object
 3   zip_code                 49993 non-null  object
 4   land_square_feet         38742 non-null  object
 5   gross_square_feet        38744 non-null  object
 6   year_built               45922 non-null  object
 7   sale_price               50000 non-null  object
 8   sale_date                50000 non-null  object
 9   latitude                 48574 non-null  object
 10  longitude                48574 non-null  object
dtypes: object(11)
memory usage: 4.2+ MB


## Classify Boroughs

In [5]:
from pkg.property_price import to_borough_name

df_land['borough'] = df_land['borough'].apply(to_borough_name)
df_land.groupby('borough').size().sort_values(ascending=False)

borough
Brooklyn         22982
Manhattan        16615
Bronx             8141
Queens            2181
Staten Island       81
dtype: int64

In [7]:
df_land_manhattan = df_land[df_land['borough']=='Manhattan']
df_land_brooklyn = df_land[df_land['borough']=='Brooklyn']
df_land_queens = df_land[df_land['borough']=='Queens']
df_land_bronx = df_land[df_land['borough']=='Bronx']
df_land_staten = df_land[df_land['borough']=='Staten Island']

In [8]:
df_land_manhattan['sale_price'] = df_land_manhattan['sale_price'].astype(float)
df_land_brooklyn['sale_price'] = df_land_brooklyn['sale_price'].astype(float)
df_land_queens['sale_price'] = df_land_queens['sale_price'].astype(float)
df_land_bronx['sale_price'] = df_land_bronx['sale_price'].astype(float)
df_land_staten['sale_price'] = df_land_staten['sale_price'].astype(float)

/var/folders/35/fnft3rfx6ys4zrh1ss8nstz40000gn/T/ipykernel_4492/2490742929.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_land_manhattan['sale_price'] = df_land_manhattan['sale_price'].astype(float)
/var/folders/35/fnft3rfx6ys4zrh1ss8nstz40000gn/T/ipykernel_4492/2490742929.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_land_brooklyn['sale_price'] = df_land_brooklyn['sale_price'].astype(float)
/var/folders/35/fnft3rfx6ys4zrh1ss8nstz40000gn/T/ipykernel_4492/2490742929.py:3: SettingWithCopyWarn

In [9]:
from pkg.property_price import to_float

df_land_manhattan['sale_price'] = to_float(df_land_manhattan['sale_price'])
df_land_brooklyn['sale_price'] = to_float(df_land_brooklyn['sale_price'])
df_land_queens['sale_price'] = to_float(df_land_queens['sale_price'])
df_land_bronx['sale_price'] = to_float(df_land_bronx['sale_price'])
df_land_staten['sale_price'] = to_float(df_land_staten['sale_price'])

/var/folders/35/fnft3rfx6ys4zrh1ss8nstz40000gn/T/ipykernel_4492/1254127301.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_land_manhattan['sale_price'] = to_float(df_land_manhattan['sale_price'])
/var/folders/35/fnft3rfx6ys4zrh1ss8nstz40000gn/T/ipykernel_4492/1254127301.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_land_brooklyn['sale_price'] = to_float(df_land_brooklyn['sale_price'])
/var/folders/35/fnft3rfx6ys4zrh1ss8nstz40000gn/T/ipykernel_4492/1254127301.py:5: SettingWithCopyWarning: 
A 

In [10]:
df_land_bronx.head()

,borough,neighborhood,building_class_category,zip_code,land_square_feet,gross_square_feet,year_built,sale_price,sale_date,latitude,longitude
239,Bronx,HUNTS POINT,31 COMMERCIAL VACANT LAND,10474,"7,500",0,NaN,0.0,2019-08-08T00:00:00.000,NaN,NaN
16090,Bronx,BATHGATE,22 STORE BUILDINGS,10457,"4,909","4,800",1931,2000000.0,2019-10-11T00:00:00.000,40.85092,-73.894496
16673,Bronx,BATHGATE,01 ONE FAMILY DWELLINGS,10457,"1,694","1,497",1899,388500.0,2019-07-23T00:00:00.000,40.853488,-73.896046
16674,Bronx,BATHGATE,01 ONE FAMILY DWELLINGS,10458,"1,095","1,260",1910,110000.0,2019-12-06T00:00:00.000,40.855949,-73.892532
16675,Bronx,BATHGATE,01 ONE FAMILY DWELLINGS,10457,"2,356","2,047",1901,470000.0,2019-03-08T00:00:00.000,40.847749,-73.896659


In [13]:
print('The number of sold properties in Manhattan:', len(df_land_manhattan))
print('The number of sold properties in Brooklyn:', len(df_land_brooklyn))
print('The number of sold properties in Queens:', len(df_land_queens))
print('The number of sold properties in Bronx:', len(df_land_bronx))
print('The number of sold properties in Staten Island:', len(df_land_staten))

The number of sold properties in Manhattan: 16615
The number of sold properties in Brooklyn: 22982
The number of sold properties in Queens: 2181
The number of sold properties in Bronx: 8141
The number of sold properties in Staten Island: 81


In [12]:
print('The mean price of sold properties in Manhattan:', df_land_manhattan['sale_price'].mean().round(2))
print('The mean price of sold properties in Brooklyn:', df_land_brooklyn['sale_price'].mean().round(2))
print('The mean price of sold properties in Queens:', df_land_queens['sale_price'].mean().round(2))
print('The mean price of sold properties in Bronx:', df_land_bronx['sale_price'].mean().round(2))
print('The mean price of sold properties in Staten Island:', df_land_staten['sale_price'].mean().round(2))

The mean price of sold properties in Manhattan: 3774714.33
The mean price of sold properties in Brooklyn: 1046619.75
The mean price of sold properties in Queens: 773099.18
The mean price of sold properties in Bronx: 671256.45
The mean price of sold properties in Staten Island: 15104.79


In [15]:
print('The maximum price of sold properties in Manhattan:', df_land_manhattan['sale_price'].max().round(2))
print('The maximum price of sold properties in Brooklyn:', df_land_brooklyn['sale_price'].max().round(2))
print('The maximum price of sold properties in Queens:', df_land_queens['sale_price'].max().round(2))
print('The maximum price of sold properties in Bronx:', df_land_bronx['sale_price'].max().round(2))
print('The maximum price of sold properties in Staten Island:', df_land_staten['sale_price'].max().round(2))

The maximum price of sold properties in Manhattan: 2155000000.0
The maximum price of sold properties in Brooklyn: 214200000.0
The maximum price of sold properties in Queens: 75000000.0
The maximum price of sold properties in Bronx: 89000000.0
The maximum price of sold properties in Staten Island: 539988.0


## Classify Years

In [30]:
df_land['sale_date'] = pd.to_datetime(df_land['sale_date'], errors='coerce')
df_land_2020 = df_land[df_land['sale_date'].dt.year >= 2020]
df_land_2020.head()


,borough,neighborhood,building_class_category,zip_code,land_square_feet,gross_square_feet,year_built,sale_price,sale_date,latitude,longitude
28,Manhattan,ALPHABET CITY,02 TWO FAMILY DWELLINGS,10009,1718,5154,1901,0.0,2020-10-02,NaN,NaN
3555,Manhattan,MIDTOWN WEST,10 COOPS - ELEVATOR APARTMENTS,10019,NaN,NaN,1923,0.0,2020-09-22,NaN,NaN
9557,Manhattan,ALPHABET CITY,03 THREE FAMILY DWELLINGS,10009,2381,3084,1899,0.0,2020-10-31,40.723816,-73.978202
18714,Manhattan,ALPHABET CITY,03 THREE FAMILY DWELLINGS,10009,2381,3084,1899,4350000.0,2020-07-16,40.723816,-73.978202
18892,Manhattan,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,10009,2021,6445,1900,0.0,2020-12-27,NaN,NaN


In [31]:
df_land_2019 = df_land[df_land['sale_date'].dt.year >= 2019]
df_land_2019.head()

,borough,neighborhood,building_class_category,zip_code,land_square_feet,gross_square_feet,year_built,sale_price,sale_date,latitude,longitude
0,Manhattan,CHELSEA,21 OFFICE BUILDINGS,10001,"7,406","40,926",1926,43300000.0,2019-03-28,40.749704,-74.00493
1,Manhattan,CHELSEA,21 OFFICE BUILDINGS,10001,"9,890","83,612",1917,148254147.0,2019-05-23,40.749364,-74.004132
2,Manhattan,CHELSEA,21 OFFICE BUILDINGS,10001,"2,498","7,380",1910,11000000.0,2019-03-13,40.752067,-74.002931
3,Manhattan,CHELSEA,21 OFFICE BUILDINGS,10011,"34,188","281,361",1936,591800000.0,2019-05-22,NaN,NaN
4,Manhattan,CHELSEA,21 OFFICE BUILDINGS,10011,"2,469","5,603",1900,0.0,2019-04-01,40.745809,-73.999729


In [33]:
df_land['sale_price'] = pd.to_numeric(df_land['sale_price'], errors='coerce') 
df_land_building = (
    df_land.groupby('building_class_category')
    .agg(
        avg_sale_price = ('sale_price', 'mean'),
        num_buildings = ('sale_price', 'count'))
    .sort_values(by='avg_sale_price', ascending=False)
)
df_land_building

,avg_sale_price,num_buildings
building_class_category,,
25 LUXURY HOTELS,7.709192e+07,18
39 TRANSPORTATION FACILITIES,7.561250e+07,2
43 CONDO OFFICE BUILDINGS,4.884568e+07,232
21 OFFICE BUILDINGS,3.246431e+07,283
26 OTHER HOTELS,1.694391e+07,38
08 RENTALS - ELEVATOR APARTMENTS,1.399861e+07,334
31 COMMERCIAL VACANT LAND,1.368122e+07,226
28 COMMERCIAL CONDOS,1.276270e+07,45
32 HOSPITAL AND HEALTH FACILITIES,1.215188e+07,41


In [34]:
df_land['sale_date'].max()

Timestamp('2020-12-27 00:00:00')